# ETL  || Planning -> Develope -> Manage & Automate

##### Source (From Where We Extract) -> We Transform (basis Our problem) -> Finally, Loading transformed data into our Target

#### Extracting Data By Running Daily This Python API Script of NIfty50 Stock !

In [1]:
import pandas as pd
from bsedata.bse import BSE  ## if not installed use: pip install bsedata
import time

In [ ]:

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

github_excel_url = "url"
engine = 'openpyxl'  ## if not installed use pip install openpyxl
EquityDF = pd.read_excel(github_excel_url, engine = engine)
EquityDF.head(n=2) # Get the list of stocks in Nifty50

,Security Code,Issuer Name,Security Id,Security Name,Status,Group,Face Value,ISIN No,Industry,Instrument,Sector Name,Industry New Name,Igroup Name,ISubgroup Name
0,500002,ABB India Limited,ABB,ABB India Limited,Active,A,2.0,INE117A01022,Heavy Electrical Equipment,Equity,Industrials,Capital Goods,Electrical Equipment,Heavy Electrical Equipment
1,500003,Aegis Logistics Ltd.,AEGISLOG,AEGIS LOGISTICS LTD.,Active,A,1.0,INE208C01025,Trading - Gas,Equity,Energy,"Oil, Gas & Consumable Fuels",Gas,Trading - Gas


In [3]:
len(EquityDF) # Just 50 stocks which are part of nifty50 index

4244

In [8]:
EquityDF[EquityDF['Security Id'] == "ADANIENT"]

,Security Code,Issuer Name,Security Id,Security Name,Status,Group,Face Value,ISIN No,Industry,Instrument,Sector Name,Industry New Name,Igroup Name,ISubgroup Name
903,512599,Adani Enterprises Ltd.,ADANIENT,ADANI ENTERPRISES LTD.,Active,A,1.0,INE423A01024,Trading - Minerals,Equity,Commodities,Metals & Mining,Metals & Minerals Trading,Trading - Minerals


##### CALLING BSE API for fetching Stocks Data for eg: Price, Code, Updated Date, Open Price, Close Price, Mrkt Cap

In [4]:
listOf_Nifty50_StockIDs = [ 
    "ADANIENT", "ADANIPORTS", "APOLLOHOSP", "ASIANPAINT", "AXISBANK",
    "BAJAJ-AUTO", "BAJFINANCE", "BAJAJFINSV", "BPCL", "BHARTIARTL",
    "BRITANNIA", "CIPLA", "COALINDIA", "DIVISLAB", "DRREDDY", "EICHERMOT",
    "GRASIM", "HCLTECH", "HDFCBANK", "HDFCLIFE", "HEROMOTOCO", "HINDALCO",
    "HINDUNILVR", "ICICIBANK", "ITC", "INDUSINDBK", "INFY", "JSWSTEEL",
    "KOTAKBANK", "LTIM", "LT", "M&M", "MARUTI", "NTPC", "NESTLEIND",
    "ONGC", "POWERGRID", "RELIANCE", "SBILIFE", "SBIN", "SUNPHARMA",
    "TCS", "TATACONSUM", "TATAMOTORS", "TATASTEEL", "TECHM", "TITAN",
    "UPL", "ULTRACEMCO", "WIPRO"
]

## We Want to filter EquityDF by looking up 'Security Id' col values exist in listOf_Nifty50_StockIDs 
EquityDF['Security Code'] = EquityDF['Security Code'].astype(str) ##coverting into str as getQuote() accepts str input

nifty50_OnlyDF= EquityDF[EquityDF['Security Id'].isin(listOf_Nifty50_StockIDs)].reset_index(drop=True)
nifty50_OnlyDF.columns = nifty50_OnlyDF.columns.str.replace(' ', '')
nifty50_OnlyDF.head()

,SecurityCode,IssuerName,SecurityId,SecurityName,Status,Group,FaceValue,ISINNo,Industry,Instrument,SectorName,IndustryNewName,IgroupName,ISubgroupName
0,500034,Bajaj Finance Limited,BAJFINANCE,Bajaj Finance Limited,Active,A,2.0,INE296A01024,Non Banking Financial Company (NBFC),Equity,Financial Services,Financial Services,Finance,Non Banking Financial Company (NBFC)
1,500087,"Cipla Ltd.,",CIPLA,CIPLA LTD.,Active,A,2.0,INE059A01026,Pharmaceuticals,Equity,Healthcare,Healthcare,Pharmaceuticals & Biotechnology,Pharmaceuticals
2,500112,"State Bank Of India,",SBIN,STATE BANK OF INDIA,Active,A,1.0,INE062A01020,Public Sector Bank,Equity,Financial Services,Financial Services,Banks,Public Sector Bank
3,500114,Titan Company Limited,TITAN,Titan Company Limited,Active,A,1.0,INE280A01028,Gems- Jewellery And Watches,Equity,Consumer Discretionary,Consumer Durables,Consumer Durables,"Gems, Jewellery And Watches"
4,500124,"Dr. Reddy's Laboratories Ltd.,",DRREDDY,DR.REDDY'S LABORATORIES LTD.,Active,A,5.0,INE089A01023,Pharmaceuticals,Equity,Healthcare,Healthcare,Pharmaceuticals & Biotechnology,Pharmaceuticals


In [5]:
nifty50_OnlyDF['SecurityCode'].values  

array(['500034', '500087', '500112', '500114', '500124', '500180',
       '500182', '500209', '500228', '500247', '500300', '500312',
       '500325', '500440', '500470', '500510', '500520', '500547',
       '500570', '500696', '500790', '500800', '500820', '500825',
       '500875', '505200', '507685', '508869', '512070', '512599',
       '524715', '532174', '532187', '532215', '532281', '532454',
       '532488', '532500', '532538', '532540', '532555', '532755',
       '532898', '532921', '532977', '532978', '533278', '540005',
       '540719', '540777'], dtype=object)

In [6]:
bseObject = BSE(update_codes=True)  ##creating Bse Lib Object

listof_stockDicts = []
sqcode_ListNifty50 = nifty50_OnlyDF['SecurityCode'].values  
for sqCode in sqcode_ListNifty50:
    try:
        stockDict = bseObject.getQuote(sqCode) #key-value pair
        stockDict.pop("buy", None)
        stockDict.pop("sell", None)
        listof_stockDicts.append(stockDict)
        time.sleep(0.5)
    except IndexError:
        print(f"IndexError for {sqCode}: Data not available")

nifty50DailyTable = pd.DataFrame(listof_stockDicts)
nifty50DailyTable.head()

,companyName,currentValue,change,pChange,updatedOn,securityID,scripCode,group,faceValue,industry,previousClose,previousOpen,dayHigh,dayLow,52weekHigh,52weekLow,weightedAvgPrice,totalTradedValue,totalTradedQuantity,2WeekAvgQuantity,marketCapFull,marketCapFreeFloat
0,Bajaj Finance Limited,8309.00,-279.50,-3.25,05 Mar 25 | 04:01 PM,BAJFINANCE,500034,A / BSE SENSEX,2.00,Financial Services,8588.50,8520.00,8520.00,8220.95,8736.00,6190.00,8326.88,51.70 Cr.,0.62 Lakh,0.29 Lakh,"5,15,058.29 Cr.","2,16,693.78 Cr."
1,CIPLA LTD.,1418.30,16.90,1.21,05 Mar 25 | 04:01 PM,CIPLA,500087,A / BSE 100,2.00,Healthcare,1401.40,1398.95,1432.15,1392.35,1702.00,1317.60,1418.09,2.48 Cr.,0.18 Lakh,0.23 Lakh,"1,14,544.03 Cr.","80,496.62 Cr."
2,STATE BANK OF INDIA,730.25,14.10,1.97,05 Mar 25 | 04:01 PM,SBIN,500112,A / BSE SENSEX,1.00,Financial Services,716.15,718.85,731.00,717.80,912.10,679.65,724.67,45.54 Cr.,6.28 Lakh,5.42 Lakh,"6,51,720.38 Cr.","2,79,904.13 Cr."
3,Titan Company Limited,3086.50,35.25,1.16,05 Mar 25 | 04:01 PM,TITAN,500114,A / BSE SENSEX,1.00,Consumer Durables,3051.25,3033.30,3105.00,3033.30,3866.15,3018.00,3082.58,7.50 Cr.,0.24 Lakh,0.19 Lakh,"2,74,015.20 Cr.","1,26,961.11 Cr."
4,DR.REDDY'S LABORATORIES LTD.,1126.55,6.35,0.57,05 Mar 25 | 04:01 PM,DRREDDY,500124,A / BSE 100,1.00,Healthcare,1120.20,1120.20,1128.85,1111.55,1420.20,1104.55,1126.03,5.79 Cr.,0.51 Lakh,1.28 Lakh,"93,999.90 Cr.","68,529.88 Cr."


#####  Data Pre-Processing (Cleaning and Transformation)  for Microsoft SQL Server Database

In [7]:
nifty50DailyTable.rename(columns={'group': 'sharegroup'}, inplace=True)
nifty50DailyTable.rename(columns={'52weekHigh': 'fiftytwoweekHigh'}, inplace=True)
nifty50DailyTable.rename(columns={'52weekLow': 'fiftytwoweekLow'}, inplace=True)
nifty50DailyTable.rename(columns={'2WeekAvgQuantity': 'twoWeekAvgQuantity'}, inplace=True)
# Convert 'updatedOn' column to datetime and extract date
nifty50DailyTable['updatedOn'] = pd.to_datetime(nifty50DailyTable['updatedOn'], format='%d %b %y | %I:%M %p', errors='coerce')

# Check if there are any invalid or missing date values
if pd.isna(nifty50DailyTable['updatedOn']).any():
    print("There are invalid or missing date values in the 'updatedOn' column.")
else:
    # Extract date from 'updatedOn' column and convert the column to datetime
    nifty50DailyTable['updatedOn'] = pd.to_datetime(nifty50DailyTable['updatedOn'].dt.date)

if 'totalTradedValueCr' not in nifty50DailyTable.columns:
   # Assuming nifty50DailyTable is your DataFrame
    nifty50DailyTable['totalTradedValueCr'] = pd.to_numeric(nifty50DailyTable['totalTradedValue'].str.replace(',', '').str.replace(' Cr.', '', regex=True), errors='coerce')  # Convert to numeric and handle 'Cr.'
    nifty50DailyTable['totalTradedQuantityLakh'] = pd.to_numeric(nifty50DailyTable['totalTradedQuantity'].str.replace(',', '').str.replace(' Lakh', '', regex=True), errors='coerce')  # Convert to numeric and handle 'Lakh'
    nifty50DailyTable['twoWeekAvgQuantityLakh'] = pd.to_numeric(nifty50DailyTable['twoWeekAvgQuantity'].str.replace(',', '').str.replace(' Lakh', '', regex=True), errors='coerce')  # Convert to numeric and handle 'Lakh'
    nifty50DailyTable['marketCapFullCr'] = pd.to_numeric(nifty50DailyTable['marketCapFull'].str.replace(',', '').str.replace(' Cr.', '', regex=True), errors='coerce')  # Convert to numeric and handle 'Cr.'
    nifty50DailyTable['marketCapFreeFloatCr'] = pd.to_numeric(nifty50DailyTable['marketCapFreeFloat'].str.replace(',', '').str.replace(' Cr.', '', regex=True), errors='coerce')  # Convert to numeric and handle 'Cr.'

    # Drop original columns
    nifty50DailyTable.drop(['totalTradedValue', 'totalTradedQuantity','twoWeekAvgQuantity', 'marketCapFull', 'marketCapFreeFloat'], axis=1, inplace=True)

nifty50DailyTable.head(n=2)

,companyName,currentValue,change,pChange,updatedOn,securityID,scripCode,sharegroup,faceValue,industry,previousClose,previousOpen,dayHigh,dayLow,fiftytwoweekHigh,fiftytwoweekLow,weightedAvgPrice,totalTradedValueCr,totalTradedQuantityLakh,twoWeekAvgQuantityLakh,marketCapFullCr,marketCapFreeFloatCr
0,Bajaj Finance Limited,8309.00,-279.50,-3.25,2025-03-05,BAJFINANCE,500034,A / BSE SENSEX,2.00,Financial Services,8588.50,8520.00,8520.00,8220.95,8736.00,6190.00,8326.88,51.70,0.62,0.29,515058.29,216693.78
1,CIPLA LTD.,1418.30,16.90,1.21,2025-03-05,CIPLA,500087,A / BSE 100,2.00,Healthcare,1401.40,1398.95,1432.15,1392.35,1702.00,1317.60,1418.09,2.48,0.18,0.23,114544.03,80496.62


#### Loading data to MySQL Relational Database Using pymysql connection and Sqlalchemy Engine

Loading Data into relational Database

In [ ]:
import pymysql
import sqlalchemy as sa
from sqlalchemy import create_engine, inspect
from sqlalchemy.exc import SQLAlchemyError

# My sql connection details
host = 'host'
database = 'database'
user = 'user'
password = 'password'
port = 'port'

# Create mysql engine
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}")
# defining function that esthablish connection with mysql with pymysql

def create_connecion():
    try:
        conn = pymysql.connect(host=host, user=user, password=password, database=database, port=port)
        cursor = conn.cursor()
        return conn, cursor
    except pymysql.Error as e:
        print(f"Error connecting mysql:{e}")
        return None, None
    
try:
    connection = engine.connect()
    print("Connection Successful!")
    connection.close()

    conn, cursor = create_connecion()
    inspector = inspect(engine)

    #Table1:nifty50_dailyData
    nifty50_table_name = 'nifty50_dailydata'

    if not inspector.has_table(nifty50_table_name):
        nifty50_table_schema = f'''
        CREATE TABLE {nifty50_table_name}(
            companyName VARCHAR(255),
            currentValue FLOAT,
            change FLOAT,
            pChange FLOAT,
            updatedOn DATE,
            securityID VARCHAR(255),
            scripCode VARCHAR(255),
            sharegroup VARCHAR(255),
            faceValue FLOAT,
            industry VARCHAR(255),
            previousClose FLOAT,
            previousOpen FLOAT,
            dayHigh FLOAT,
            dayLow FLOAT,
            fiftytwoweekHigh FLOAT,
            fiftytwoweekLow FLOAT,
            weightedAvgPrice FLOAT,
            totalTradedQuantityLakh FLOAT,
            totalTradedValueCr FLOAT,
            twoWeekAvgQuantityLakh FLOAT,
            marketCapFullCr FLOAT,
            marketCapFreeFloatCr FLOAT
        );
        '''
        cursor.execute(nifty50_table_schema)
        conn.commit()
        print(f"Table {nifty50_table_name} created successfully!")

    conn.close()
    
    with engine.begin() as engineConn:
        sql_max_updatedOn = pd.read_sql_query(sa.text(f'SELECT MAX(updatedOn) FROM {nifty50_table_name}'), engineConn).iloc[0, 0]
        print("Latest date in SQL:", sql_max_updatedOn)

        df_max_updatedOn = nifty50DailyTable['updatedOn'].max()
        print("Latest date in DataFrame:", df_max_updatedOn)

        if pd.isnull(sql_max_updatedOn) and not pd.isnull(df_max_updatedOn):
            nifty50DailyTable.to_sql(nifty50_table_name, engine, index=False, if_exists='append', method='multi')
            print("Daily Data inserted successfully.")
        elif df_max_updatedOn > pd.Timestamp(sql_max_updatedOn):
            nifty50DailyTable.to_sql(nifty50_table_name, engine, index=False, if_exists='append', method='multi')
            print("New data appended successfully.")
        else:
            print("No new data to append.")

    
    company_table_name = 'nifty50_companydata'

    if not inspector.has_table(company_table_name):
        company_table_schema = f'''
        CREATE TABLE {company_table_name} (
            securityCode VARCHAR(255),
            issuerName VARCHAR(255),
            securityId VARCHAR(255),
            securityName VARCHAR(255),
            status VARCHAR(255),
            CompanyGroup VARCHAR(255),
            faceValue FLOAT,
            isinNo VARCHAR(255),
            industry VARCHAR(255),
            instrument VARCHAR(255),
            sectorName VARCHAR(255),
            industryNewName VARCHAR(255),
            igroupName VARCHAR(255),
            iSubgroupName VARCHAR(255)
        );
        '''
        conn, cursor = create_connection()
        cursor.execute(company_table_schema)
        conn.commit()
        conn.close()
        print(f"Table {company_table_name} created successfully!")

        nifty50_OnlyDF.rename(columns={'Group': 'CompanyGroup'}, inplace=True)
        nifty50_OnlyDF.to_sql(company_table_name, engine, index=False, if_exists='append', method='multi')
        print("Company Data inserted successfully.")
    else:
        print("Company Table already exists, skipping creation.")



except SQLAlchemyError as e:
    print(f"Error:{e}")

